# 表情符号生成器（Emojify!）

在本节中，你将学习如何利用**词向量表示（word vector representations）**来构建一个“表情符号生成器”（Emojifier）。


### 项目目标

你是否曾经希望自己的短信更生动、更有表现力？这个表情符号生成器应用程序（emojifier app）将帮助你实现这一点。

例如，将下面这句话：

> “Congratulations on the promotion! Lets get coffee and talk. Love you!”

自动变成：

> “Congratulations on the promotion! 👍 Lets get coffee and talk. ☕️ Love you! ❤️”

### 任务说明

你将实现一个模型，它能够输入一句话（例如 “Let's go see the baseball game tonight!”），并输出最合适的表情符号（⚾️）。

在许多表情界面中，你必须记住 ❤️ 是“heart”（心形）符号，而不是“love”（爱）符号。但通过使用**词向量**，你会看到即使训练集中仅明确地将少数单词与某个表情符号关联起来，你的算法仍然能够**泛化（generalize）**到测试集中的其他单词上——即便这些单词在训练集中从未出现过。

这种特性让你能够在**小规模训练集**的基础上，依然构建一个准确的“句子到表情符号”的分类器（classifier）。


### 模型版本

在本练习中，你将依次构建两个模型：

1. **Emojifier-V1**：一个使用词嵌入（word embeddings）的基础模型。
2. **Emojifier-V2**：一个进一步结合 **LSTM（长短期记忆网络）** 的高级模型。


### 准备开始

现在，让我们开始吧！运行以下代码单元以加载所需的包。


In [2]:
#!pip install emoji

   ---------------------------------------- 0.0/608.4 kB ? eta -:--:--
   - ------------------------------------- 20.5/608.4 kB 682.7 kB/s eta 0:00:01
   --------- ------------------------------ 143.4/608.4 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  604.2/608.4 kB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 608.4/608.4 kB 4.8 MB/s eta 0:00:00


In [1]:
# ----------------------------
# 导入常用库
# ----------------------------

import numpy as np
# numpy：Python 的科学计算库，提供高效数组操作与线性代数函数

import emoji
# emoji：用于处理文本中的表情符号，例如识别、替换或解析表情

import csv
# csv：用于读取和写入 CSV 文件

import pandas as pd
# pandas：Python 数据分析库，提供高效的 DataFrame 数据结构

import matplotlib.pyplot as plt
# matplotlib.pyplot：绘图库，用于绘制折线图、散点图、柱状图等

from sklearn.metrics import confusion_matrix
# sklearn.metrics.confusion_matrix：用于计算分类模型的混淆矩阵

# Jupyter Notebook 专用设置
%matplotlib inline
# 在 Jupyter Notebook 中直接显示 matplotlib 绘制的图，而不是弹出新窗口


### 辅助函数

In [3]:
# ----------------------------
# 读取 GloVe 词向量函数
# ----------------------------
def read_glove_vecs(glove_file):
    """
    功能：
    从 GloVe 文件读取词向量，并构建三种映射：
        1. words_to_index: 单词 -> 索引（整数）
        2. index_to_words: 索引 -> 单词（反向映射）
        3. word_to_vec_map: 单词 -> 词向量（numpy 数组）
    
    参数：
    - glove_file : str
        GloVe 文件路径，例如 "glove.6B.50d.txt"
        文件格式：
            每行由单词和其向量组成，空格分隔，例如：
                king 0.2134 0.1153 ... -0.4123
    
    返回：
    - words_to_index : dict
        单词 -> 索引（索引从1开始，0可留作PAD）
    - index_to_words : dict
        索引 -> 单词
    - word_to_vec_map : dict
        单词 -> numpy 向量（dtype=float64）
    """

    # 使用 UTF-8 编码打开文件
    with open(glove_file, 'r', encoding='utf-8') as f:

        # 使用 set 来存储所有单词（自动去重）
        words = set()                 # 初始化空集合，用于存储词表
        # 字典用于存储单词 -> 向量的映射
        word_to_vec_map = {}          

        # 遍历 GloVe 文件中的每一行
        for line in f:
            # 去除行首行尾空格，并按空格拆分成列表
            line = line.strip().split()  

            # 第一个元素是单词本身
            curr_word = line[0]  

            # 将单词加入集合（去重）
            words.add(curr_word)  

            # 剩余部分是向量，将其转换为 numpy 数组
            # dtype=np.float64 保证高精度
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)

        # ----------------------------
        # 构建单词索引映射
        # ----------------------------
        # 索引从 1 开始，通常 0 留作 PAD
        i = 1
        words_to_index = {}   # 单词 -> 索引
        index_to_words = {}   # 索引 -> 单词（反向映射）

        # 按字典序排序单词，保证索引固定
        for w in sorted(words):  
            words_to_index[w] = i   # 给单词分配索引
            index_to_words[i] = w   # 建立索引 -> 单词映射
            i += 1                  # 索引递增

    # 返回三个映射
    return words_to_index, index_to_words, word_to_vec_map


In [4]:
# ----------------------------
# Softmax 函数
# ----------------------------
def softmax(x):
    """
    功能：
    对输入向量或矩阵计算 softmax 概率分布。
    softmax 常用于多分类模型的输出，将任意实数映射到 (0,1) 区间，并且总和为1。
    
    参数：
    - x : numpy.ndarray
        输入向量或矩阵，可以是一维向量或二维矩阵
        1D 示例: [2.0, 1.0, 0.1]
        2D 示例: [[2.0, 1.0, 0.1], [1.2, 0.7, 0.5]]
    
    返回：
    - softmax 输出 : numpy.ndarray
        与 x 形状相同，但每行（或整向量）被映射为概率分布，总和为1
    """
    
    # ----------------------------
    # 数值稳定的指数计算
    # ----------------------------
    # np.max(x) 返回 x 中的最大值
    # x - np.max(x) 可以防止指数函数 exp() 计算时溢出
    # 例如 exp(1000) 会溢出，用 x - max(x) 可以避免
    e_x = np.exp(x - np.max(x))  
    
    # ----------------------------
    # softmax 公式：exp(x_i)/sum(exp(x_j))
    # ----------------------------
    # 将每个元素除以所有元素的和，得到概率分布
    return e_x / e_x.sum()  


In [5]:
# ----------------------------
# 读取 CSV 数据集
# ----------------------------
def read_csv(filename='data/emojify_data.csv'):
    """
    功能：
    从 CSV 文件中读取句子和对应的 emoji 标签，用于表情符号分类任务。
    
    参数：
    - filename : str, 可选，默认 'data/emojify_data.csv'
        CSV 文件路径，文件内容一般为两列：
            第一列：文本句子（string）
            第二列：对应的 emoji 标签（整数或类别）
    
    返回：
    - X : numpy.ndarray
        包含所有句子的数组，形状为 (num_samples,)
    
    - Y : numpy.ndarray
        包含对应标签的数组，形状为 (num_samples,)
    """
    
    # 初始化列表，用于存放句子和标签
    phrase = []    # 存放句子
    emoji = []     # 存放对应的 emoji 标签

    # 打开 CSV 文件
    with open(filename) as csvDataFile:
        # 创建 CSV 阅读器对象
        csvReader = csv.reader(csvDataFile)
        
        # 遍历 CSV 中的每一行
        for row in csvReader:
            phrase.append(row[0])        # row[0] 是句子，加入 phrase 列表
            emoji.append(row[1])         # row[1] 是标签，加入 emoji 列表

    # 将列表转换为 numpy 数组，便于后续处理
    X = np.asarray(phrase)                     # 句子数组，dtype 默认为 string
    Y = np.asarray(emoji, dtype=int)          # 标签数组，dtype 指定为 int

    # 返回句子数组和标签数组
    return X, Y


In [6]:
# ----------------------------
# 转换标签为 one-hot 编码
# ----------------------------
def convert_to_one_hot(Y, C):
    """
    功能：
    将整数标签数组 Y 转换为 one-hot 编码形式，适合分类任务的神经网络输入。
    
    参数：
    - Y : numpy.ndarray
        标签数组，形状为 (m,) 或 (m,1)
        每个元素是一个整数，表示类别索引，例如 0,1,2,...,C-1
    - C : int
        类别总数，例如有 5 个类别，C=5
    
    返回：
    - one-hot 编码 : numpy.ndarray
        形状为 (m, C)
        每一行表示一个样本的 one-hot 编码，例如：
            Y[i] = 2, C=5 -> [0,0,1,0,0]
    """

    # np.eye(C) 生成一个 C x C 的单位矩阵
    # 每一行就是一个类别的 one-hot 向量
    # Y.reshape(-1) 将 Y 展平为一维数组，保证索引正确
    Y = np.eye(C)[Y.reshape(-1)]

    # 返回转换后的 one-hot 编码
    return Y


In [7]:
# ----------------------------
# emoji 映射字典
# ----------------------------
emoji_dictionary = {
    "0": "\u2764\uFE0F",   # ❤️ 心，Unicode 表示法 \u2764 是红心符号，\uFE0F 是变体选择器，用于显示 emoji 样式
    "1": ":baseball:",     # ⚾ 棒球，这里使用了简写格式 ":baseball:"，通常在某些 Markdown 或 Slack 中可自动渲染为棒球 emoji
    "2": ":smile:",        # 😄 微笑，表示开心或高兴的表情
    "3": ":disappointed:", # 😞 失望，用于表示沮丧或不满
    "4": ":fork_and_knife:"# 🍴 餐具，表示与吃饭或食物相关
}

# 注解：
# - 键 "0"-"4" 对应整数标签，通常与模型输出类别索引一致
# - 值为对应的 emoji 字符或简写，可在显示预测结果时使用
# - Unicode 形式可确保在各种平台上都能显示标准 emoji


In [8]:
# ----------------------------
# 标签转 emoji
# ----------------------------
def label_to_emoji(label):
    """
    将整数标签或字符串标签转为 emoji 字符

    参数：
        label -- 标签，可以是整数（如 0,1,2,...）或者字符串（如 "0","1",...）
                 对应模型输出的类别索引

    返回：
        emoji 字符串 -- 对应标签的 emoji 表示，例如 ❤️, ⚾, 😄 等
    """

    # 将标签转换为字符串，用于在 emoji_dictionary 中查找对应的 emoji
    # emoji_dictionary[str(label)] 返回对应的 emoji 简写或 Unicode
    # emoji.emojize() 将 emoji 简写（如 ":baseball:"）转换为可显示的 emoji 字符
    return emoji.emojize(emoji_dictionary[str(label)])


In [9]:
# ----------------------------
# 打印预测结果
# ----------------------------
def print_predictions(X, pred):
    """
    打印每个句子及对应的预测 emoji

    参数：
        X    -- 句子数组，形状为 (m,)，m 是句子数量
                例如：["I love you", "Baseball is fun", ...]
        pred -- 模型预测标签数组，形状为 (m, 1) 或 (m,)
                每个元素是整数标签（如 0,1,2,...），对应 emoji 类别
    """

    # 打印空行，美观
    print()

    # 遍历每个句子
    for i in range(X.shape[0]):
        # X[i]：当前句子
        # pred[i]：对应预测标签，使用 int() 转为整数
        # label_to_emoji()：将标签转换为 emoji
        # print()：打印句子和对应的 emoji
        print(X[i], label_to_emoji(int(pred[i])))


In [10]:
# ----------------------------
# 绘制混淆矩阵
# ----------------------------
def plot_confusion_matrix(y_actu, y_pred, title='Confusion matrix', cmap=plt.cm.gray_r):
    """
    绘制混淆矩阵，用于可视化模型预测结果与真实标签的对比

    参数：
        y_actu -- 真实标签数组，形状 (m,)，每个元素为类别整数
        y_pred -- 模型预测标签数组，形状 (m,1) 或 (m,)，每个元素为预测类别整数
        title  -- 图表标题，默认为 'Confusion matrix'
        cmap   -- 颜色映射，默认使用灰色反转 colormap (plt.cm.gray_r)
    """

    # --------------------------------------------
    # 使用 pandas 计算交叉表（混淆矩阵的原始数据）
    # --------------------------------------------
    df_confusion = pd.crosstab(
        y_actu,                         # 行：真实标签
        y_pred.reshape(y_pred.shape[0],), # 列：预测标签，将 y_pred 展平为 1D 数组
        rownames=['Actual'],            # 行名称
        colnames=['Predicted'],         # 列名称
        margins=True                    # 添加行/列总计
    )

    # --------------------------------------------
    # 归一化混淆矩阵（每行除以总和）
    # 便于显示比例而非绝对数值
    # --------------------------------------------
    df_conf_norm = df_confusion / df_confusion.sum(axis=1)

    # --------------------------------------------
    # 绘制矩阵图
    # plt.matshow() 绘制矩阵形式的图像
    # --------------------------------------------
    plt.matshow(df_confusion, cmap=cmap)  # 使用指定 colormap
    plt.colorbar()                        # 添加颜色条

    # --------------------------------------------
    # 设置坐标轴刻度
    # tick_marks = 类别索引列表
    # --------------------------------------------
    tick_marks = np.arange(len(df_confusion.columns))  # 列索引
    plt.xticks(tick_marks, df_confusion.columns, rotation=45) # x轴刻度及旋转
    plt.yticks(tick_marks, df_confusion.index)                # y轴刻度

    # --------------------------------------------
    # 设置坐标轴标签
    # --------------------------------------------
    plt.ylabel(df_confusion.index.name)    # y轴标签为 'Actual'
    plt.xlabel(df_confusion.columns.name)  # x轴标签为 'Predicted'

    # --------------------------------------------
    # 设置图标题
    # --------------------------------------------
    plt.title(title)
    plt.show()  # 显示图像


In [11]:
# ----------------------------
# 预测函数
# ----------------------------
def predict(X, Y, W, b, word_to_vec_map):
    """
    给定句子 X、权重矩阵 W 和偏置 b，预测对应的 emoji 标签，并计算预测准确率

    参数：
        X              -- 输入句子数组，形状 (m,)，每个元素为字符串句子
        Y              -- 标签数组，形状 (m,1)，每个元素为整数类别
        W              -- 权重矩阵，形状 (num_classes, word_vector_dim)，用于线性层
        b              -- 偏置向量，形状 (num_classes,1)
        word_to_vec_map -- 字典，单词 -> 词向量映射

    返回：
        pred -- 预测标签数组，形状 (m,1)，每个元素为预测类别索引
    """

    m = X.shape[0]                    # 样本数量，即句子数量
    pred = np.zeros((m, 1))           # 初始化预测数组，全为 0，形状 (m,1)

    # 遍历每个样本句子
    for j in range(m):
        words = X[j].lower().split()  # 将句子转为小写并拆分为单词列表
        avg = np.zeros((50,))         # 初始化平均词向量（假设每个词向量维度为 50）
        
        # 遍历句子中每个单词
        for w in words:
            avg += word_to_vec_map[w] # 累加单词向量

        avg = avg / len(words)        # 计算句子的平均词向量（句子级表示）

        # ----------------------------
        # 线性层计算
        # ----------------------------
        Z = np.dot(W, avg) + b        # 线性变换: Z = W * avg + b
                                      # Z 形状为 (num_classes,1)

        A = softmax(Z)                # 计算概率分布，softmax 将 Z 转换为每个类别的概率
                                      # A 形状为 (num_classes,1)，所有值加和为 1

        pred[j] = np.argmax(A)        # 选择概率最大的类别索引作为预测标签
                                      # np.argmax 返回沿指定轴最大值的索引，这里为预测类别

    # ----------------------------
    # 计算并输出准确率
    # ----------------------------
    accuracy = np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))  # 与真实标签比较
    print("Accuracy: " + str(accuracy))

    return pred  # 返回预测标签数组，形状 (m,1)


## 1 - 基线模型：Emojifier-V1

### 1.1 - 数据集 EMOJISET

让我们从构建一个简单的基线分类器开始。

在这个实验中，你将使用一个非常小的数据集 **(X, Y)**，其中：

- **X** 包含 **127 条句子（字符串）**  
- **Y** 为每条句子对应的 **整数标签（0 到 4 之间）**，每个标签代表一种表情符号


<img src="images/data_set.png" style="width:700px;height:300px;">
<caption><center> **图 1**：EMOJISET —— 一个包含 5 个类别的分类问题。上图展示了部分句子示例。</center></caption>


接下来，让我们使用以下代码加载该数据集。  
我们将数据集划分为：
- **训练集（127 个样本）**
- **测试集（56 个样本）**

（以下代码将在后续单元中执行）


In [12]:
# ----------------------------
# 读取训练集和测试集
# ----------------------------

# 读取训练集 CSV 数据
# X_train: 训练句子数组，形状 (m_train,)
# Y_train: 训练标签数组，形状 (m_train,)
X_train, Y_train = read_csv('data/train_emoji.csv')

# 读取测试集 CSV 数据
# X_test: 测试句子数组，形状 (m_test,)
# Y_test: 测试标签数组，形状 (m_test,)
X_test, Y_test = read_csv('data/tesss.csv')

# 输出样本数量与前几个样本，便于检查
print("训练集样本数量:", X_train.shape[0])
print("测试集样本数量:", X_test.shape[0])
print("训练集前5条样本句子:", X_train[:5])
print("训练集前5条样本标签:", Y_train[:5])


训练集样本数量: 132
测试集样本数量: 56
训练集前5条样本句子: ['never talk to me again' 'I am proud of your achievements'
 'It is the worst day in my life' 'Miss you so much' 'food is life']
训练集前5条样本标签: [3 2 3 0 4]


In [14]:
# ----------------------------
# 计算训练集中句子的最大长度
# ----------------------------

# max(X_train, key=len)  
# - 找到训练集中最长的句子（按字符数比较）
# .split()  
# - 将该句子按空格拆分为单词列表
# len(...)  
# - 计算该句子的单词数
# maxLen: 训练集中最长句子的单词数，用于之后句子向量化和填充
maxLen = len(max(X_train, key=len).split())

print("训练集中最长句子的单词数:", maxLen)


训练集中最长句子的单词数: 10


运行以下代码单元，以打印出来自 **X_train** 的句子及其在 **Y_train** 中对应的标签。  
你可以修改变量 `index` 的值，以查看不同的示例。

>  注意：由于 iPython Notebook 使用的字体原因，心形表情符号 ❤️ 可能会显示为黑色（🖤），这是正常现象。

```python
# 示例代码
index = 10  # 可修改以查看不同样本
print(X_train[index], label_to_emoji(Y_train[index]))


In [15]:
# ----------------------------
# 查看训练集中的某个样本及其对应的 emoji 标签
# ----------------------------

# 设置要查看的样本索引
index = 45

# 打印训练集 X_train 中该索引对应的句子
# 同时调用 label_to_emoji() 函数将 Y_train 中的标签转换为 emoji
print(X_train[index], label_to_emoji(Y_train[index]))

# 说明：
# X_train[index] -> 第 45 个训练样本（句子）
# Y_train[index] -> 第 45 个训练样本对应的标签（数字 0~4）
# label_to_emoji(...) -> 将数字标签映射为对应 emoji，便于可视化


you two are cute ❤️


### 1.2 - Emojifier-V1 模型概览

在本部分中，你将实现一个基线模型，名为 **“Emojifier-V1”**。


<center>
<img src="images/image_1.png" style="width:900px;height:300px;">
<caption><center>**图 2**：基线模型（Emojifier-V1）。</center></caption>
</center>


该模型的输入是一个代表句子的字符串（例如：“I love you”）。  
模型的输出是一个形状为 **(1, 5)** 的**概率向量（probability vector）**，表示该句子属于五个表情类别中每一类的概率。

最终，你将把这个概率向量传入一个 **argmax 层**，以提取出**最可能的表情符号对应的索引**，也就是模型预测的结果。


为了让标签 **Y** 适用于 **Softmax 分类器（softmax classifier）** 的训练，我们需要将它从当前的形状  
$(m, 1)$ 转换为 **“独热表示”（one-hot representation）**，形状为 $(m, 5)$。

在这种表示方式中，每一行是一个 **one-hot 向量**，用于指示某个样本所属的类别。

例如：
- 若标签为 2，则对应的 one-hot 向量为 `[0, 0, 1, 0, 0]`。

下面的代码片段可以完成这一转换操作。  
其中，变量名 `Y_oh_train` 和 `Y_oh_test` 中的 `Y_oh` 表示 **“Y 的 one-hot 形式（Y-one-hot）”**。

```python
# 将标签转换为 one-hot 向量表示
Y_oh_train = convert_to_one_hot(Y_train, C = 5)
Y_oh_test = convert_to_one_hot(Y_test, C = 5)


In [16]:
# ----------------------------
# 将标签转换为独热编码（One-Hot Encoding）
# ----------------------------

# 将训练集标签 Y_train 转换为独热编码形式
# 参数说明：
#   Y_train : 原始标签数组，形状 (m, )，每个值为 0~4 之间的整数
#   C       : 类别总数，这里是 5 个 emoji
# 返回：
#   Y_oh_train : 训练集标签的独热编码，形状 (m, 5)
Y_oh_train = convert_to_one_hot(Y_train, C = 5)

# 将测试集标签 Y_test 转换为独热编码形式
# 参数说明同上
Y_oh_test = convert_to_one_hot(Y_test, C = 5)

# 说明：
# 1. 独热编码的好处：
#    - 可以直接用于神经网络的输出层训练
#    - 将类别标签从标量转换为向量，例如：
#      标签 2 → [0, 0, 1, 0, 0]
# 2. convert_to_one_hot 函数内部通过 np.eye(C)[Y.reshape(-1)] 实现


让我们来看一下 `convert_to_one_hot()` 函数的运行结果。  
你可以修改变量 `index` 的值，以打印并查看不同样本对应的 **one-hot 向量表示**。

```python
# 查看某个样本的 one-hot 转换结果
index = 1  # 可更改索引以查看不同样本
print(f"标签值: {Y_train[index]}")
print(f"One-hot 表示: {Y_oh_train[index]}")


In [17]:
# ----------------------------
# 查看某个样本的标签及其独热编码
# ----------------------------

# 设置要查看的样本索引
index = 25

# 打印该样本的原始标签以及对应的独热编码
# Y_train[index]       : 原始标签（整数），表示第 index 个训练样本的 emoji 类别
# Y_oh_train[index]    : 独热编码向量，对应类别位置为 1，其余位置为 0
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

# 示例说明：
# 假设 Y_train[25] = 3，则输出可能为：
# 3 is converted into one hot [0. 0. 0. 1. 0.]
# 这里 [0. 0. 0. 1. 0.] 表示独热编码形式


1 is converted into one hot [0. 1. 0. 0. 0.]


现在，所有数据都已经准备完毕，可以输入到 **Emojifier-V1 模型** 中进行训练了。  
接下来，让我们开始实现这个模型吧！ 🚀

```python
# 实现 Emojifier-V1 模型的函数框架
def model(X, Y, word_to_vec_map, learning_rate=0.01, num_iterations=400):
    """
    实现 Emojifier-V1 模型的训练过程。
    
    参数：
    X -- 句子列表（每个元素为字符串）
    Y -- 句子对应的标签（0~4）
    word_to_vec_map -- 词向量字典：每个单词映射到其对应的 GloVe 向量
    learning_rate -- 学习率（默认 0.01）
    num_iterations -- 迭代次数（默认 400）

    返回：
    pred -- 模型对所有样本的预测结果
    """
    # 模型实现细节将在后续小节中完成
    pass


### 1.3 - 实现 Emojifier-V1

如图（2）所示，第一步是将输入的句子转换为词向量表示，然后对这些词向量取平均值。  
与之前的练习类似，我们将使用预训练的 **50 维 GloVe 词向量**。  
运行以下代码单元以加载包含所有词向量表示的 `word_to_vec_map`。


In [18]:
# ----------------------------
# 读取 GloVe 词向量
# ----------------------------

# 调用之前定义的 read_glove_vecs() 函数，读取 GloVe 文件
# 'data/glove.6B.50d.txt' ：GloVe 预训练词向量文件路径，50维度词向量
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

# 返回值说明：
# word_to_index : dict，将单词映射为对应的索引
#                 例如: word_to_index['king'] = 1234
# index_to_word : dict，将索引映射回单词
#                 例如: index_to_word[1234] = 'king'
# word_to_vec_map : dict，将单词映射为对应的词向量（numpy.ndarray）
#                 例如: word_to_vec_map['king'].shape = (50,) 表示 50 维向量

# 读取完成后，可用于：
# 1. 将单词转换为向量进行运算
# 2. 在神经网络中作为输入的嵌入向量
# 3. 进行相似度计算、类比任务等


你已经加载了以下内容：

- `word_to_index`：将单词映射到词汇表索引的字典（共 400,001 个单词，有效索引范围 0 到 400,000）  
- `index_to_word`：将索引映射回词汇表对应单词的字典  
- `word_to_vec_map`：将单词映射到其 GloVe 向量表示的字典  

运行以下代码单元以检查加载是否正常。


In [19]:
# ----------------------------
# 演示单词索引与词向量映射
# ----------------------------

# 定义要查询的单词
word = "cucumber"

# 定义要查询的索引
index = 289846

# 1. 查询单词对应的索引
# word_to_index[word] 会返回单词在词汇表中的索引
print("the index of", word, "in the vocabulary is", word_to_index[word])
# 输出示例: the index of cucumber in the vocabulary is 1234

# 2. 查询索引对应的单词
# index_to_word[index] 会返回索引对应的单词
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])
# 输出示例: the 289846th word in the vocabulary is cucumber

# 说明：
# - word_to_index 用于将单词转为索引，方便在神经网络中查找嵌入向量
# - index_to_word 用于将索引转回单词，便于解释模型输出或调试


the index of cucumber in the vocabulary is 113317
the 289846th word in the vocabulary is potatos


**练习**：实现 `sentence_to_avg()` 函数。你需要完成以下两步：

1. 将每个句子转换为小写字母，然后将句子拆分为单词列表。可以使用 `X.lower()` 和 `X.split()`。  
2. 对句子中的每个单词，获取其对应的 GloVe 向量表示。然后，对所有词向量取平均值。


In [20]:
# ----------------------------
# 将句子转换为平均词向量
# ----------------------------
def sentence_to_avg(sentence, word_to_vec_map):
    """
    功能：
        将一句话（sentence）转换为单词列表，提取每个单词的 GloVe 词向量，并计算平均向量。
        最终返回一个表示整句话语义的向量。
    
    参数：
        sentence -- 字符串类型，一条训练样本（例如 X_train 中的一条句子）
        word_to_vec_map -- 字典类型，将词汇表中每个单词映射为 50 维词向量
        
    返回：
        avg -- 句子的平均词向量，numpy 数组，形状为 (50,)
    """
    
    # ----------------------------
    # 1. 将句子拆分成单词列表，并全部转换为小写
    # 例如 "I love NLP" -> ["i", "love", "nlp"]
    # ----------------------------
    words = sentence.lower().split()
    
    # ----------------------------
    # 2. 初始化一个全零的向量，用于累加词向量
    # 维度与 GloVe 向量相同，这里是 50 维
    # ----------------------------
    avg = np.zeros(50,)
    
    # ----------------------------
    # 3. 遍历句子中的每个单词，将对应词向量累加
    # ----------------------------
    for w in words:
        avg += word_to_vec_map[w]   # 累加每个单词的 50 维词向量
    
    # ----------------------------
    # 4. 将累加的向量除以单词数，得到平均向量
    # ----------------------------
    avg = np.divide(avg, len(words))   # avg = avg / len(words)
    
    # ----------------------------
    # 5. 返回句子的平均向量
    # ----------------------------
    return avg


In [21]:
# ----------------------------
# 示例：将一句话转换为平均词向量
# ----------------------------

# 句子示例
sentence = "Morrocan couscous is my favorite dish"

# 调用 sentence_to_avg 函数，将句子转换为 50 维平均词向量
avg = sentence_to_avg(sentence, word_to_vec_map)

# 打印结果
print("avg = ", avg)


avg =  [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


**Expected Output**:

<table>
    <tr>
        <td>
            **avg= **
        </td>
        <td>
           [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]
        </td>
    </tr>
</table>

#### 模型

现在，你已经具备完成 `model()` 函数实现的所有组件。  
在使用 `sentence_to_avg()` 后，你需要将平均向量输入到前向传播中，计算损失，然后进行反向传播以更新 softmax 参数。

**练习**：实现图（2）中描述的 `model()` 函数。  
假设这里的 $Yoh$（“Y one hot”）是输出标签的 one-hot 编码，前向传播和交叉熵损失计算公式如下：

$$ z^{(i)} = W . avg^{(i)} + b$$  
$$ a^{(i)} = softmax(z^{(i)})$$  
$$ \mathcal{L}^{(i)} = - \sum_{k = 0}^{n_y - 1} Yoh^{(i)}_k * log(a^{(i)}_k)$$

当然可以实现一个更高效的向量化实现。但由于我们这次仍然使用 for 循环将句子逐一转换为 $avg^{(i)}$ 表示，这里就不再优化了。

我们已经为你提供了一个 `softmax()` 函数。


In [13]:
# ----------------------------
# GRADED FUNCTION: model
# ----------------------------
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    基于平均词向量的简单神经网络模型，用于训练词向量表示和 emoji 预测
    
    参数：
    X -- 输入数据，numpy 数组，每个元素是一个句子（字符串），形状 (m, 1)
    Y -- 标签，numpy 数组，整数形式（0~4 对应 emoji 类别），形状 (m, 1)
    word_to_vec_map -- 字典，将词映射到 50 维 GloVe 向量
    learning_rate -- 学习率，用于随机梯度下降
    num_iterations -- 迭代次数
    
    返回：
    pred -- 模型预测结果，numpy 数组，形状 (m, 1)
    W -- softmax 层权重矩阵，形状 (n_y, n_h)
    b -- softmax 层偏置，形状 (n_y,)
    """
    
    np.random.seed(1)  # 设置随机种子，保证结果可复现

    # ----------------------------
    # 初始化参数
    # ----------------------------
    m = Y.shape[0]      # 训练样本数量
    n_y = 5             # 分类类别数（emoji 类别）
    n_h = 50            # GloVe 向量维度
    
    # Xavier 初始化权重矩阵 W，使权重分布更均匀
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)  
    b = np.zeros((n_y,))  # 偏置向量初始化为 0

    # 将标签 Y 转换为 one-hot 编码，便于 softmax 计算
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # ----------------------------
    # 梯度下降优化循环
    # ----------------------------
    for t in range(num_iterations):            # 遍历迭代次数
        for i in range(m):                     # 遍历每个训练样本
            
            # ----------------------------
            # 第 i 个训练样本：计算句子平均词向量
            # ----------------------------
            avg = sentence_to_avg(X[i], word_to_vec_map)  # avg 形状 (50,)

            # ----------------------------
            # 前向传播
            # z = W·avg + b
            # ----------------------------
            z = np.dot(W, avg) + b                        # 线性组合，形状 (n_y,)
            a = softmax(z)                                # softmax 输出概率分布

            # ----------------------------
            # 计算损失（交叉熵损失）
            # ----------------------------
            cost = -np.sum(Y_oh[i]*np.log(a))             # 第 i 个样本的损失

            # ----------------------------
            # 反向传播计算梯度
            # ----------------------------
            dz = a - Y_oh[i]                              # softmax 误差
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))  # 权重梯度
            db = dz                                       # 偏置梯度

            # ----------------------------
            # 参数更新（随机梯度下降）
            # ----------------------------
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        # 每 100 个 epoch 输出一次损失和准确率，便于观察训练效果
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)  # 输出当前预测准确率

    # 返回最终预测、权重和偏置
    return pred, W, b


In [14]:
# 输出训练集 X_train 的形状
print(X_train.shape)
# X_train 是句子列表，通常形状为 (m, ) 或 (m,1)，m = 样本数

# 输出训练标签 Y_train 的形状
print(Y_train.shape)
# Y_train 是整数标签数组，形状 (m, 1)

# 将 Y_train 转为 one-hot 编码并查看形状
print(np.eye(5)[Y_train.reshape(-1)].shape)
# np.eye(5) 生成 5x5 单位矩阵
# Y_train.reshape(-1) 将 Y_train 拉成一维
# np.eye(5)[Y_train.reshape(-1)] 得到每个标签对应的 one-hot 编码
# 结果形状 (m, 5)

# 打印训练集第一个样本
print(X_train[0])
# 输出示例句子字符串

# 查看 X_train 的数据类型
print(type(X_train))
# 通常是 numpy.ndarray 或 list

# 构建一个小型标签数组 Y（16 个样本）
Y = np.asarray([5,0,0,5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
print(Y.shape)
# 输出 Y 的形状 (20,)

# 构建一个小型句子数组 X，对应上面的标签 Y
X = np.asarray([
    'I am going to the bar tonight', 
    'I love you', 
    'miss you my dear',
    'Lets go party and drinks',
    'Congrats on the new job',
    'Congratulations',
    'I am so happy for you',
    'Why are you feeling bad',
    'What is wrong with you',
    'You totally deserve this prize',
    'Let us go play football',
    'Are you down for football this afternoon',
    'Work hard play harder',
    'It is suprising how people can be dumb sometimes',
    'I am very disappointed',
    'It is the best day in my life',
    'I think I will end up alone',
    'My life is so boring',
    'Good job',
    'Great so awesome'
])
print(X.shape)
# 输出 X 的形状 (20,)

# 再次打印 Y_train 的 one-hot 编码形状
print(np.eye(5)[Y_train.reshape(-1)].shape)

# 再次打印 X_train 的类型
print(type(X_train))


(132,)
(132,)
(132, 5)
never talk to me again
<class 'numpy.ndarray'>
(20,)
(20,)
(132, 5)
<class 'numpy.ndarray'>


运行下一个代码单元以训练你的模型，并学习 softmax 参数 **(W, b)**。


In [15]:
# 训练模型
pred, W, b = model(X_train, Y_train, word_to_vec_map)
# model(...) 返回三个值：
# pred -- numpy 数组，模型对训练集 X_train 的预测标签，形状 (m, 1)
# W    -- softmax 层的权重矩阵，形状 (n_y, n_h)，这里 n_y=5（类别数），n_h=50（GloVe 向量维度）
# b    -- softmax 层的偏置向量，形状 (n_y,)

# 输出训练后的预测结果
print(pred)
# pred 的每个元素是训练集中每个句子的预测 emoji 类别（整数索引）


Epoch: 0 --- cost = 1.9520498812810072
Accuracy: 0.3484848484848485
Epoch: 100 --- cost = 0.07971818726014807
Accuracy: 0.9318181818181818
Epoch: 200 --- cost = 0.04456369243681402
Accuracy: 0.9545454545454546
Epoch: 300 --- cost = 0.03432267378786059
Accuracy: 0.9696969696969697
[[3.]
 [2.]
 [3.]
 [0.]
 [4.]
 [0.]
 [3.]
 [2.]
 [3.]
 [1.]
 [3.]
 [3.]
 [1.]
 [3.]
 [2.]
 [3.]
 [2.]
 [3.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [1.]
 [4.]
 [3.]
 [3.]
 [4.]
 [0.]
 [3.]
 [4.]
 [2.]
 [0.]
 [3.]
 [2.]
 [2.]
 [3.]
 [4.]
 [2.]
 [2.]
 [0.]
 [2.]
 [3.]
 [0.]
 [3.]
 [2.]
 [4.]
 [3.]
 [0.]
 [3.]
 [3.]
 [3.]
 [4.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [3.]
 [1.]
 [0.]
 [0.]
 [0.]
 [3.]
 [4.]
 [4.]
 [2.]
 [2.]
 [1.]
 [2.]
 [0.]
 [3.]
 [2.]
 [2.]
 [0.]
 [3.]
 [3.]
 [1.]
 [2.]
 [1.]
 [2.]
 [2.]
 [4.]
 [3.]
 [3.]
 [2.]
 [4.]
 [0.]
 [0.]
 [3.]
 [3.]
 [3.]
 [3.]
 [2.]
 [0.]
 [1.]
 [2.]
 [3.]
 [0.]
 [2.]
 [2.]
 [2.]
 [3.]
 [2.]
 [2.]
 [2.]
 [4.]
 [1.]
 [1.]
 [3.]
 [3.]
 [4.]
 [1.]
 [2.]
 [1.]
 [1.]
 [3.]
 [1.]

**Expected Output** (on a subset of iterations):

<table>
    <tr>
        <td>
            **Epoch: 0**
        </td>
        <td>
           cost = 1.95204988128
        </td>
        <td>
           Accuracy: 0.348484848485
        </td>
    </tr>


<tr>
        <td>
            **Epoch: 100**
        </td>
        <td>
           cost = 0.0797181872601
        </td>
        <td>
           Accuracy: 0.931818181818
        </td>
    </tr>
    
<tr>
        <td>
            **Epoch: 200**
        </td>
        <td>
           cost = 0.0445636924368
        </td>
        <td>
           Accuracy: 0.954545454545
        </td>
    </tr>
    
    <tr>
        <td>
            **Epoch: 300**
        </td>
        <td>
           cost = 0.0343226737879
        </td>
        <td>
           Accuracy: 0.969696969697
        </td>
    </tr>
</table>

太棒了！你的模型在训练集上的准确率相当高。  
现在让我们看看它在测试集上的表现如何。


### 1.4 - 测试集性能评估


In [16]:
# ----------------------------
# 对训练集进行预测
# ----------------------------
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
# 调用 predict 函数：
# X_train -- 输入句子列表
# Y_train -- 真实标签
# W, b -- 模型训练得到的 softmax 参数
# word_to_vec_map -- GloVe 词向量字典
# 返回 pred_train：训练集的预测结果，形状 (m, 1)
# 函数内部会：
#   1. 将每个句子拆分成单词
#   2. 取每个单词的 GloVe 向量并求平均得到句子向量
#   3. 计算 Z = W·avg + b
#   4. 通过 softmax 得到概率向量
#   5. 取最大概率索引作为预测
#   6. 计算预测准确率并打印

# ----------------------------
# 对测试集进行预测
# ----------------------------
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)
# 用训练好的 W 和 b 对测试集 X_test 进行预测
# 返回 pred_test：测试集的预测结果，形状 (m_test, 1)
# 同样内部会打印测试集的准确率


Training set:
Accuracy: 0.9772727272727273
Test set:
Accuracy: 0.8571428571428571


**Expected Output**:

<table>
    <tr>
        <td>
            **Train set accuracy**
        </td>
        <td>
           97.7
        </td>
    </tr>
    <tr>
        <td>
            **Test set accuracy**
        </td>
        <td>
           85.7
        </td>
    </tr>
</table>

在有 5 个类别的情况下，随机猜测的准确率只有 20%。  
而在仅用 127 个样本训练后，你的模型就能达到这样的性能，已经相当不错了。

在训练集中，算法见过句子 "*I love you*"，对应标签为 ❤️。  
然而，你可以注意到单词 "adore" 并未出现在训练集中。  
尽管如此，让我们看看输入 "*I adore you*" 时会发生什么。


In [17]:
# ----------------------------
# 自定义句子及对应标签
# ----------------------------
X_my_sentences = np.array([
    "i adore you",             # 句子1
    "i love you",              # 句子2
    "funny lol",               # 句子3
    "lets play with a ball",   # 句子4
    "food is ready",           # 句子5
    "not feeling happy"        # 句子6
])

Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])
# 每个句子的真实标签
# 0: ❤️ (heart)
# 1: ⚾ (baseball)
# 2: 😄 (smile)
# 3: 😞 (disappointed)
# 4: 🍴 (fork_and_knife)

# ----------------------------
# 对自定义句子进行预测
# ----------------------------
pred = predict(X_my_sentences, Y_my_labels, W, b, word_to_vec_map)
# predict 函数内部步骤：
# 1. 遍历每个句子，将句子拆成单词
# 2. 对每个单词取 GloVe 词向量
# 3. 求句子所有词向量的平均，得到句子向量 avg
# 4. 前向传播计算 Z = W · avg + b
# 5. 通过 softmax 得到概率分布
# 6. 取概率最大值对应索引作为预测结果
# 7. 打印预测的准确率

# ----------------------------
# 打印预测结果
# ----------------------------
print_predictions(X_my_sentences, pred)
# print_predictions 函数：
# 遍历每个句子
# 调用 label_to_emoji(pred[i]) 将预测的标签索引转换为 emoji
# 打印 "句子 → emoji"
# 例如输出：
# i adore you ❤️
# funny lol 😄


Accuracy: 0.8333333333333334

i adore you ❤️
i love you ❤️
funny lol :smile:
lets play with a ball ⚾
food is ready 🍴
not feeling happy :smile:


C:\Users\luxia\AppData\Local\Temp\ipykernel_33892\3157549296.py:129: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(X[i], label_to_emoji(int(pred[i])))


太棒了！由于 *adore* 与 *love* 的词向量相似，算法能够正确泛化，即使是一个从未见过的单词也能处理正确。  
像 *heart*、*dear*、*beloved* 或 *adore* 这样的词，其词向量都与 *love* 类似，因此也可能同样有效——你可以自由修改上述输入，尝试各种句子，看看效果如何。

需要注意的是，算法无法正确处理 "*not feeling happy*" 这种情况。  
这是因为该算法忽略了**单词顺序**，因此不擅长理解诸如 "*not happy*" 这样的短语。

打印 **混淆矩阵（confusion matrix）** 也能帮助理解哪些类别对你的模型来说更难。  
混淆矩阵显示了某个实际标签为某类别（"actual" class）的样本，被算法误标为其他类别（"predicted" class"）的频率。


In [18]:
# ----------------------------
# 查看测试集标签的形状
# ----------------------------
print(Y_test.shape)
# 输出 (56,) → 测试集有 56 个句子

# ----------------------------
# 打印列标题（emoji 表示每个类别）
# ----------------------------
print('           ' + label_to_emoji(0) + '    ' + label_to_emoji(1) + '    ' + label_to_emoji(2) + '    ' + label_to_emoji(3) + '   ' + label_to_emoji(4))
# label_to_emoji 函数将类别索引转换为 emoji
# 输出示例：           ❤️    ⚾    😄    😞   🍴

# ----------------------------
# 生成测试集混淆矩阵
# ----------------------------
pd.crosstab(
    Y_test,                        # 行：真实标签
    pred_test.reshape(56,),        # 列：预测标签
    rownames=['Actual'],           # 行名
    colnames=['Predicted'],        # 列名
    margins=True                   # 添加行/列总计
)
# 输出一个 DataFrame，显示每个真实类别被预测为各类别的次数
# margins=True 会在最后一行/列显示总和

# ----------------------------
# 可视化混淆矩阵
# ----------------------------
plot_confusion_matrix(Y_test, pred_test)
# plot_confusion_matrix 函数步骤：
# 1. 用 pd.crosstab 构建混淆矩阵
# 2. 归一化（除以每行总和）得到每个类别预测比例
# 3. plt.matshow 显示矩阵，颜色表示预测数量
# 4. 设置坐标轴标签、标题、颜色条
# 5. 输出图像方便直观分析模型预测情况


(56,)
           ❤️    ⚾    :smile:    :disappointed:   🍴


<font color='blue'>
    
**本部分需要记住的内容**：
- 即使只有 127 个训练样本，你也可以得到一个相当不错的表情符号生成模型。这要归功于词向量提供的泛化能力。  
- Emojify-V1 在处理像 *"This movie is not good and not enjoyable"* 这样的句子时表现不佳，因为它无法理解单词组合——它只是将所有单词的词向量平均，而不关注单词的顺序。你将在下一部分构建一个更好的算法。


## 2 - Emojifier-V2：在Pytorch中使用 LSTM

让我们构建一个 **LSTM 模型**，输入为单词序列。  
该模型能够考虑单词的顺序。Emojifier-V2 将继续使用**预训练词向量**来表示单词，但会将这些词向量输入到 LSTM 中，由 LSTM 预测最合适的表情符号。

运行以下代码单元以加载Pytorch所需的包。


In [19]:
# ----------------------------
# 导入常用库
# ----------------------------
import numpy as np  
# 导入 NumPy 库，用于高效的数值计算、数组操作和线性代数运算

import torch  
# 导入 PyTorch 主库，用于张量操作、自动求导和深度学习模型构建

import torch.nn as nn  
# 导入 torch.nn 模块，其中包含构建神经网络的各类层（如 Linear, Conv2d, LSTM 等）
# nn 是构建模型的基础模块

import torch.nn.functional as F  
# 导入 torch.nn.functional 模块，提供了大量函数化的操作，如激活函数（relu, softmax 等）、损失函数等
# 与 nn.Module 中的层不同，functional 提供的是“函数式”操作，不包含可学习参数

from torch.utils.data import Dataset, DataLoader  
# Dataset：用于封装自定义数据集，必须实现 __len__() 和 __getitem__() 方法
# DataLoader：提供批量读取数据、打乱顺序和多线程加载的功能
# 常用于训练循环中按 batch 获取数据

import random  
# Python 标准库的 random 模块，用于生成随机数、随机选择或打乱数据
# 在训练或初始化权重时常用


In [20]:
# ----------------------------
# 设备选择（CPU / GPU）
# ----------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
# torch.device()：PyTorch 中表示存放张量和模型的设备对象
# "cuda"：表示 GPU，如果有可用的 NVIDIA GPU，可用于加速计算
# "cpu"：表示使用 CPU 进行计算
# torch.cuda.is_available()：检查当前系统是否有可用的 GPU
# 语句含义：如果有可用 GPU，则使用 GPU；否则退回到 CPU
# 将 device 对象传给张量或模型，可以实现张量/模型在 GPU 上运算


### 2.1 - 模型概览（PyTorch 版本）

下面是你将要实现的 **Emojifier-V2** PyTorch 版本：

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> **图 3**：Emojifier-V2。一个两层 LSTM 序列分类器。</center></caption>

在 PyTorch 中，我们将使用 `nn.Embedding`、`nn.LSTM` 和 `nn.Linear` 来实现这个模型。  
训练方式也将使用 mini-batch，PyTorch 中要求同一 mini-batch 的所有序列长度相同，这与 Keras 相同。


### 2.2 Mini-batch 与填充（Padding）

在 PyTorch 中处理不同长度的序列时也必须进行 **填充（padding）**。  
具体方法与 Keras 相同：设定最大句子长度 `max_len`，将所有序列填充或截断到相同长度。例如：
因此，句子 "i love you" 将表示为：
$$(e_{i}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$$

其中 0 表示填充值，长度统一为 `max_len`。  
这使得序列可以同时被送入 LSTM，便于批量计算。

在 PyTorch 中，可以通过自定义函数 `sentences_to_indices()` 将句子转换为索引矩阵。

### 2.3 - 嵌入层（Embedding layer）

在 PyTorch 中，嵌入矩阵由 `nn.Embedding` 表示，它将正整数（单词索引）映射到固定大小的稠密向量（embedding 向量）。  
我们可以使用预训练的 GloVe 词向量初始化该层，并将其设置为 **不可训练（freeze=True）**，保持词向量固定。

`Embedding()` 层的输入是一个整数矩阵，形状为 **(batch size, max input length)**。  
这对应于将句子转换为索引列表（整数），如下图所示：

<img src="images/embedding1.png" style="width:700px;height:250px;">
<caption><center> **图 4**：嵌入层示意图。该示例展示了两个样本通过嵌入层的传播过程。  
两个样本都被用 0 填充至长度 `max_len=5`。最终表示的维度为 `(2, max_len, 50)`，因为使用的词向量为 50 维。</center></caption>

输入中的最大整数（即单词索引）不应大于词汇表大小。  
该层输出形状为 **(batch size, max input length, 词向量维度)**。

第一步是将所有训练句子转换为索引列表，然后对这些列表进行 **零填充（zero-padding）**，使其长度等于最长句子的长度。

实现下列函数，将 **X**（句子数组，字符串形式）转换为对应单词索引的数组。  
输出形状应适用于 `Embedding()` 层（如图 4 所示）。

In [21]:
# ----------------------------
# 函数：sentences_to_indices
# ----------------------------
def sentences_to_indices(X, word_to_index, max_len):
    """
    功能：
        将文本句子转换为索引矩阵，每个单词用词表中的索引表示。
        用于将文本数据输入到神经网络模型（如 Embedding 层）中。
    
    参数：
        X           -- 输入句子列表，长度为 m，每个元素是一个字符串（句子）
        word_to_index -- 字典，将单词映射为索引。例如 {'i':1, 'love':2, ...}
        max_len     -- 每个句子的最大长度，超过的单词截断，不足的句子补 0
    
    返回：
        X_indices   -- numpy 数组，形状为 (m, max_len)，每行是句子对应的单词索引
    """

    # ----------------------------
    # 1. 获取句子数量 m
    # ----------------------------
    m = len(X)  # len(X) 返回句子总数

    # ----------------------------
    # 2. 初始化索引矩阵 X_indices
    # ----------------------------
    X_indices = np.zeros((m, max_len))  
    # 使用全 0 初始化
    # shape: (句子数量 m, 每句最大长度 max_len)
    # 如果句子长度不足 max_len，自动补 0

    # ----------------------------
    # 3. 遍历每个句子
    # ----------------------------
    for i, sentence in enumerate(X):  # enumerate 返回索引 i 和句子内容 sentence
        words = sentence.lower().split()  # 将句子小写并拆分为单词列表

        # ----------------------------
        # 4. 遍历句子中的单词
        # ----------------------------
        for j, w in enumerate(words):  # j: 单词位置, w: 单词
            if j >= max_len:  # 如果句子长度超过 max_len，则截断
                break

            # 使用 word_to_index.get() 获取单词索引
            # 如果单词不在词表中，返回 0
            X_indices[i, j] = word_to_index.get(w, 0)

    # ----------------------------
    # 5. 返回索引矩阵
    # ----------------------------
    return X_indices


运行以下代码单元，以查看 `sentences_to_indices()` 的作用，并检查你的结果。


In [22]:
# 输入句子列表
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])

# 调用 sentences_to_indices 函数，将句子转成索引矩阵
X1_indices = sentences_to_indices(X1, word_to_index, max_len=5)

# 输出句子和对应的索引矩阵
print("X1 =", X1)
print("X1_indices =", X1_indices)


X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[155345. 225122.      0.      0.      0.]
 [220930. 286375.  69714.      0.      0.]
 [151204. 192973. 302254. 151349. 394475.]]


**Expected Output**:

<table>
    <tr>
        <td>
            **X1 =**
        </td>
        <td>
           ['funny lol' 'lets play football' 'food is ready for you']
        </td>
    </tr>
    <tr>
        <td>
            **X1_indices =**
        </td>
        <td>
           [[ 155345.  225122.       0.       0.       0.] <br>
            [ 220930.  286375.  151266.       0.       0.] <br>
            [ 151204.  192973.  302254.  151349.  394475.]]
        </td>
    </tr>
</table>

让我们在 **PyTorch** 中构建 `nn.Embedding` 层，使用预训练的词向量。  
在构建好该层后，你将把 `sentences_to_indices()` 的输出作为输入传入，Embedding 层将返回句子的词向量表示。

实现 `pretrained_embedding_layer()` 的步骤如下：

1. 初始化嵌入矩阵为全零的 numpy 数组，形状为 `(vocab_len, embedding_dim)`。  
2. 遍历 `word_to_vec_map`，将每个词对应的 GloVe 向量填入嵌入矩阵的正确行。  
3. 定义 PyTorch 的 `nn.Embedding` 层，并使用 `nn.Embedding.from_pretrained()` 将嵌入矩阵加载进去：  
   - 设置 `freeze=True`，保证词向量在训练中保持固定。  
   - 若设置 `freeze=False`，训练时嵌入向量会被更新。  
4. 返回构建好的 Embedding 层。


In [23]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    功能：
        创建一个 PyTorch 的 Embedding 层，并用预训练的词向量（如 GloVe）初始化。
        该层可以直接用于神经网络输入，将单词索引映射为词向量。
    
    参数：
        word_to_vec_map (dict): 字典，将每个单词映射到其预训练词向量，例如 GloVe
                                 格式: {'word': np.array([50维向量])}
        word_to_index (dict): 字典，将每个单词映射到唯一索引
                              格式: {'word': index}
    
    返回：
        embedding_layer (nn.Embedding): PyTorch Embedding 层，权重已经用预训练词向量初始化
                                        并且参数冻结 (freeze=True)，不可训练
    """

    # ----------------------------
    # 1. 获取词汇表长度 + 1
    # ----------------------------
    # vocab_len: 词汇表大小 + 1，因为索引通常从1开始，0保留给padding
    vocab_len = len(word_to_index) + 1

    # ----------------------------
    # 2. 获取词向量维度
    # ----------------------------
    # emb_dim: 词向量的维度，例如 GloVe 50维、100维等
    emb_dim = list(word_to_vec_map.values())[0].shape[0]

    # ----------------------------
    # 3. 初始化词向量矩阵
    # ----------------------------
    # emb_matrix: 形状为 (vocab_len, emb_dim)
    # 第 i 行存储索引 i 对应的词向量
    emb_matrix = np.zeros((vocab_len, emb_dim))

    # ----------------------------
    # 4. 将预训练词向量填入矩阵
    # ----------------------------
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]  # 第 idx 行就是 word 的词向量

    # ----------------------------
    # 5. 转为 PyTorch 张量
    # ----------------------------
    emb_matrix = torch.tensor(emb_matrix, dtype=torch.float32)

    # ----------------------------
    # 6. 创建 PyTorch Embedding 层
    # ----------------------------
    # from_pretrained: 用给定权重初始化 Embedding
    # freeze=True: 冻结权重，不在训练中更新
    embedding_layer = nn.Embedding.from_pretrained(emb_matrix, freeze=True)

    return embedding_layer


In [24]:
# ----------------------------
# 1. 创建预训练嵌入层
# ----------------------------
# 调用之前定义的函数，用 GloVe 词向量初始化 Embedding 层
# embedding_layer: nn.Embedding 类型，权重已冻结
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

# ----------------------------
# 2. 访问嵌入矩阵权重
# ----------------------------
# embedding_layer.weight 返回 Embedding 层的权重张量
# detach(): 将张量从计算图中分离，不追踪梯度
# numpy(): 转为 NumPy 数组，方便索引或打印
weights = embedding_layer.weight.detach().numpy()  # 形状: (vocab_len, emb_dim)

# ----------------------------
# 3. 打印指定索引的词向量元素
# ----------------------------
# 例如：weights[1][3]
# - weights[1] 表示词汇表索引为 1 的单词的词向量
# - weights[1][3] 表示该词向量的第 4 个维度的值
print("weights[0][1][3] =", weights[1][3])


weights[0][1][3] = -0.3403


**Expected Output**:

<table>
    <tr>
        <td>
            **weights[0][1][3] =**
        </td>
        <td>
           -0.3403
        </td>
    </tr>
</table>

## 2.3 构建 Emojifier-V2

现在让我们构建 **Emojifier-V2** 模型。  
你将使用之前构建的嵌入层，并将其输出传入两层 LSTM 网络，最后通过全连接层输出 softmax 概率。

<img src="images/emojifier-v2.png" style="width:700px;height:400px;"> <br>
<caption><center> **图 3**：Emojifier-V2。一个两层 LSTM 序列分类器。</center></caption>

---

在 PyTorch 中，我们用 `nn.Module` 构建该模型，主要结构如下：

- **输入**：句子索引矩阵，形状为 `(batch_size, max_len)`  
- **Embedding 层**：将每个单词索引映射为其 GloVe 词向量 `(batch_size, max_len, emb_dim)`  
- **LSTM 层 1**：隐藏状态大小 128，返回整序列 `(batch_size, max_len, 128)`  
- **Dropout 1**：防止过拟合，概率 0.5  
- **LSTM 层 2**：隐藏状态大小 128，只返回最后一个隐藏状态 `(batch_size, 128)`  
- **Dropout 2**：概率 0.5  
- **全连接层**：输出大小为 `C=5`  
- **Softmax 激活**：输出每个类别的概率

In [25]:
# ---------------------------- Emojify-V2 模型 ---------------------------- #
class EmojifyV2(nn.Module):  
    """
    双层 LSTM 模型，用于根据输入句子预测对应的 Emoji 表情类别。
    模型结构：
    1. 预训练嵌入层（使用 GloVe 向量）
    2. 双层 LSTM（128 单元）
    3. Dropout 正则化
    4. 全连接层 + Softmax 输出分类
    """
    
    def __init__(self, max_len, word_to_vec_map, word_to_index, C=5):
        """
        初始化模型结构。
        
        参数：
        max_len -- int，输入句子的最大长度（用于 Padding）
        word_to_vec_map -- dict，词到词向量的映射（GloVe）
        word_to_index -- dict，词到索引的映射
        C -- int，分类数（例如：5 类表情）
        """
        # 继承父类构造函数
        super(EmojifyV2, self).__init__()
        
        # 保存最大序列长度
        self.max_len = max_len
        
        # ---------------------------- 1️ 预训练嵌入层 ----------------------------
        # 使用预训练的 GloVe 词向量，固定不训练（freeze=True）
        # 每个词被映射为一个 embedding 向量（例如 50 维）
        self.embedding = pretrained_embedding_layer(word_to_vec_map, word_to_index)
        
        # ---------------------------- 2️ 第一层 LSTM ----------------------------
        # input_size：每个时间步输入向量的维度（即 embedding_dim）
        # hidden_size：LSTM 隐藏状态的维度（此处为 128）
        # batch_first=True：输入输出的第一个维度为 batch
        self.lstm1 = nn.LSTM(
            input_size=self.embedding.embedding_dim,  
            hidden_size=128,
            batch_first=True
        )
        
        # Dropout 用于防止过拟合
        self.dropout1 = nn.Dropout(0.5)
        
        # ---------------------------- 3️ 第二层 LSTM ----------------------------
        # 接收前一层的输出，继续提取时序特征
        self.lstm2 = nn.LSTM(
            input_size=128,  
            hidden_size=128,
            batch_first=True
        )
        
        self.dropout2 = nn.Dropout(0.5)
        
        # ---------------------------- 4️ 全连接输出层 ----------------------------
        # 输入维度 128（LSTM 隐层输出），输出维度为类别数 C
        self.fc = nn.Linear(128, C)
    
    
    def forward(self, X):
        """
        前向传播逻辑：将输入句子索引序列映射为类别概率。
        
        参数：
        X -- 张量，形状 (batch_size, max_len)，包含词索引序列
        
        返回：
        每个样本的分类概率，形状 (batch_size, C)
        """
        # Step 1: 词嵌入
        emb = self.embedding(X.long())  
        # emb 形状: (batch_size, max_len, emb_dim)
        
        # Step 2: 第一层 LSTM
        lstm_out1, _ = self.lstm1(emb)  
        # 输出 lstm_out1 形状: (batch_size, max_len, 128)
        
        lstm_out1 = self.dropout1(lstm_out1)
        
        # Step 3: 第二层 LSTM
        lstm_out2, (h_n, _) = self.lstm2(lstm_out1)
        # h_n 形状: (num_layers=1, batch_size, hidden_size=128)
        
        # Step 4: 取最后一层 LSTM 的最后一个隐藏状态
        lstm_out2 = self.dropout2(h_n[-1])  
        # lstm_out2 形状: (batch_size, 128)
        
        # Step 5: 全连接层 + Softmax
        out = self.fc(lstm_out2)  
        # out 形状: (batch_size, C)
        
        return F.softmax(out, dim=1)


In [26]:
# ---------------------------- 训练函数（带进度显示） ---------------------------- #
from tqdm import tqdm   # tqdm 用于在训练过程中显示动态进度条

def train(model, X_train_indices, Y_train_oh, num_epochs=50, batch_size=32, lr=0.001):
    """
    模型训练函数（带进度显示与性能评估）

    参数说明：
    model           -- PyTorch 模型对象（如 EmojifyV2 实例）
    X_train_indices -- 训练集输入（已转为词索引的 numpy 数组，形状为 (m, maxLen)）
    Y_train_oh      -- 训练集标签的 one-hot 编码（形状为 (m, C)）
    num_epochs      -- 训练轮数（默认 50）
    batch_size      -- 每批次样本数（默认 32）
    lr              -- 学习率（默认 0.001）
    """

    # ---------------------------- 1️⃣ 优化器与损失函数 ----------------------------
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  
    # 使用 Adam 优化器更新参数，能自动调节学习率
    
    criterion = nn.CrossEntropyLoss()  
    # 交叉熵损失函数：用于多分类问题（内部自动做 softmax）

    # ---------------------------- 2️⃣ 构建数据加载器 ----------------------------
    dataset = torch.utils.data.TensorDataset(
        torch.tensor(X_train_indices, dtype=torch.long),                   # 输入索引张量
        torch.tensor(np.argmax(Y_train_oh, axis=1), dtype=torch.long)      # 标签转为类别索引
    )

    # DataLoader 会自动打乱（shuffle=True）并按批（batch_size）加载数据
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 将模型放到设备上（GPU 或 CPU）
    model.to(device)

    # ---------------------------- 3️⃣ 训练循环 ----------------------------
    for epoch in range(num_epochs):
        model.train()      # 启用训练模式（Dropout 生效）
        total_loss = 0.0   # 累计损失
        correct = 0        # 预测正确样本数
        total = 0          # 总样本数

        # tqdm 用于显示当前 epoch 的训练进度条
        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

        for X_batch, Y_batch in pbar:
            # 将当前批数据加载到 GPU 或 CPU
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

            # ------------------ 前向传播 ------------------
            out = model(X_batch)          # 模型预测输出 (batch_size, C)
            loss = criterion(out, Y_batch)  # 计算交叉熵损失

            # ------------------ 反向传播 ------------------
            optimizer.zero_grad()   # 清空上一步的梯度
            loss.backward()         # 反向传播，计算每个参数的梯度
            optimizer.step()        # 更新参数（根据梯度下降）

            # ------------------ 统计性能指标 ------------------
            total_loss += loss.item()                         # 累加本批次的损失
            preds = torch.argmax(out, dim=1)                  # 获取预测类别
            correct += (preds == Y_batch).sum().item()        # 累加正确预测数
            total += Y_batch.size(0)                          # 累加样本总数

            # tqdm 实时显示当前批次的 loss 与准确率
            pbar.set_postfix({
                "loss": f"{loss.item():.4f}",
                "acc": f"{(correct / total):.4f}"
            })

        # ---------------------------- 4️⃣ 每个 epoch 打印汇总 ----------------------------
        avg_loss = total_loss / len(loader)  # 平均损失
        acc = correct / total                # 平均准确率

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"{len(loader)}/{len(loader)} [==============================] "
              f"- loss: {avg_loss:.4f} - acc: {acc:.4f}")


In [27]:
# ---------------------------- 测试评估函数 ---------------------------- #
def evaluate(model, X_test_indices, Y_test_oh):
    """
    模型评估函数（evaluation function）
    作用：在测试集上计算模型预测准确率
    
    参数说明：
    - model: 训练好的 PyTorch 模型
    - X_test_indices: 测试集输入索引（numpy 数组）
    - Y_test_oh: 测试集标签的 one-hot 编码
    """
    
    # 将模型切换为“评估模式”（evaluation mode）
    # 作用：关闭 Dropout、BatchNorm 等训练时的随机行为
    model.eval()
    
    # 将 numpy 数组转换为 LongTensor 并放到设备上（CPU 或 GPU）
    X_test_tensor = torch.tensor(X_test_indices, dtype=torch.long).to(device)
    
    # 将 one-hot 标签转换为类别索引（使用 argmax 取最大概率对应的类）
    Y_test_tensor = torch.tensor(np.argmax(Y_test_oh, axis=1), dtype=torch.long).to(device)
    
    # 关闭梯度计算，加快推理速度、减少显存占用
    with torch.no_grad():
        # 前向传播，得到模型输出（每个样本在各类别上的分数）
        pred = model(X_test_tensor)
        
        # 从每个样本的预测分数中，取最大值对应的类别作为预测结果
        pred_labels = torch.argmax(pred, dim=1)
        
        # 计算预测准确率：预测标签与真实标签相等的比例
        acc = (pred_labels == Y_test_tensor).float().mean().item()
    
    # 打印准确率结果
    print("Test accuracy =", acc)
    
    # 将预测结果从 GPU 移到 CPU 并转为 numpy 数组返回，方便后续分析或展示
    return pred_labels.cpu().numpy()


In [28]:
# ---------------------------- 打印预测函数 ---------------------------- #
def print_predictions(model, X_test, X_test_indices, Y_test, label_to_emoji):
    """
    打印预测错误样本的函数
    
    功能：
    在测试集上运行模型预测，并输出预测错误的样本，
    包含原文本、真实表情与预测表情，用于人工检查模型效果。
    
    参数说明：
    - model: 训练好的 PyTorch 模型
    - X_test: 原始测试集样本（通常是句子文本列表）
    - X_test_indices: 测试集索引形式输入（模型输入格式）
    - Y_test: 测试集真实标签（非 one-hot 形式）
    - label_to_emoji: 将标签编号转换为表情符号的函数
    """

    # 设置为评估模式（关闭 Dropout / BatchNorm 的随机行为）
    model.eval()
    
    # 将测试集索引数据转为 Tensor，并移动到对应设备上（CPU 或 GPU）
    X_test_tensor = torch.tensor(X_test_indices, dtype=torch.long).to(device)

    # 关闭梯度计算，加快预测速度
    with torch.no_grad():
        # 前向传播得到预测结果（每个样本的类别得分）
        pred = model(X_test_tensor)
        
        # 取每个样本的最大得分类别（即预测结果）
        pred_labels = torch.argmax(pred, dim=1).cpu().numpy()

    # 打印提示信息
    print(f"错误信息如下（仅展示预测错误的样本）:")
    print("-" * 60)

    # 遍历测试样本
    for i in range(len(X_test)):
        # 判断预测是否错误
        if pred_labels[i] != Y_test[i]:
            # 打印真实表情 + 原始文本 + 预测表情
            print(f"✅ 正确表情：{label_to_emoji(Y_test[i])}   "
                  f"❌ 预测结果：{X_test[i]} {label_to_emoji(pred_labels[i]).strip()}")


你的 Emojifier-V2 **模型**输入为形状 (`m`, `max_len`) 的数组，输出为形状 (`m`, 类别数) 的概率向量。  

因此，我们需要做两件事：

1. 将训练集句子从字符串形式转换为索引形式，方便嵌入层处理：

```python
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
```

2. 将标签从整数索引转换为 one-hot 编码，以便用于交叉熵损失：
```python
Y_train_oh = np.eye(C)[Y_train.reshape(-1)]
Y_test_oh = np.eye(C)[Y_test.reshape(-1)]
```

这样处理后：

- X_train_indices 和 X_test_indices 的形状为 (样本数, max_len)，每个元素是对应单词的索引

- Y_train_oh 和 Y_test_oh 的形状为 (样本数, C)，每行是标签的 one-hot 编码


In [29]:
# ========================================
# 数据预处理：句子索引化与标签 One-hot 编码
# ========================================

# 假设以下变量已在前面定义好：
# X_train, Y_train, X_test, Y_test               —— 训练集与测试集（输入句子与标签）
# word_to_index                                  —— 单词到索引的映射字典
# word_to_vec_map                                —— 单词到词向量的映射字典
# maxLen                                         —— 每个句子的最大长度（例如 10）
# 这些数据通常来自前面的文本处理与 GloVe 向量加载步骤。

# -------------------------------------------------
# C = 5 表示分类任务中类别的总数（即表情类别数）
# 在表情任务中共有 5 种类别，对应 5 种 emoji 表情。
# -------------------------------------------------
C = 5


# =================================================
# 1. 将训练集中的每个句子转换为词索引矩阵
# =================================================
# sentences_to_indices() 函数作用：
#   - 输入：一组句子 + 单词索引字典 + 句子最大长度 maxLen
#   - 输出：二维数组（形状为 (m, maxLen)），
#           每一行是一个句子的单词索引序列（不足 maxLen 用 0 填充）
# 参数：
#   X_train       —— 训练集文本句子（numpy 数组）
#   word_to_index —— 单词与索引的映射字典，如 {"i":1, "love":2, "food":3, ...}
#   maxLen        —— 每个句子统一长度，超出部分截断，不足部分补 0
# 返回：
#   X_train_indices —— 训练集中每个句子的索引化结果
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)


# =================================================
# 2. 将测试集中的句子转换为相同格式的索引矩阵
# =================================================
# 参数与返回结果与训练集相同
# 这样模型在训练与测试阶段的输入格式完全一致。
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)


# =================================================
# 3. 将标签（Y_train）转换为 One-hot 向量
# =================================================
# np.eye(C) —— 创建一个 C×C 的单位矩阵（对角线为 1，其余为 0）
# 例如 C=5 时：
# np.eye(5) =
# [[1,0,0,0,0],
#  [0,1,0,0,0],
#  [0,0,1,0,0],
#  [0,0,0,1,0],
#  [0,0,0,0,1]]
#
# Y_train.reshape(-1) —— 将标签 Y_train 展平成一维数组
# 例如原本形状是 (132, 1)，reshape(-1) 变为 (132,)
# 这样 np.eye(C)[Y_train.reshape(-1)] 会为每个标签取对应行，
# 生成形状为 (m, C) 的 One-hot 矩阵。
Y_train_oh = np.eye(C)[Y_train.reshape(-1)]


# =================================================
# 4. 将测试集标签也转换为 One-hot 向量
# =================================================
# 操作与训练集完全相同，用于后续模型评估。
Y_test_oh = np.eye(C)[Y_test.reshape(-1)]


运行以下代码单元以创建模型并查看其摘要。  
由于数据集中所有句子长度都小于 10 个单词，我们选择 `max_len = 10`。  

你应该可以看到模型架构，它使用了 **20,224,951** 个参数，其中 **20,000,050**（词向量部分）为不可训练，其余 **224,901** 个为可训练参数。  
由于我们的词汇表大小为 400,001 个单词（有效索引从 0 到 400,000），因此不可训练参数数量为 400,001 × 50 = 20,000,050。


In [30]:
# ========================================
# 模型初始化：EmojifyV2 模型创建
# ========================================

# -------------------------------------------------
# maxLen = 10
# -------------------------------------------------
# 定义每个输入句子的最大长度（max sequence length）。
# 模型在处理输入时，每个句子都会被转换为长度为 10 的单词索引序列：
#   - 若句子长度 < 10，则用 0（padding）填充；
#   - 若句子长度 > 10，则截断多余部分。
# 这样可以保证所有输入样本的形状一致，便于批量训练。
maxLen = 10


# -------------------------------------------------
# 模型实例化
# -------------------------------------------------
# 创建 EmojifyV2 模型对象，并将其加载到指定的计算设备上（CPU 或 GPU）。
# 参数说明：
#   maxLen            —— 输入句子的最大长度，用于确定嵌入层输入维度
#   word_to_vec_map   —— 单词到 GloVe 词向量的映射字典
#   word_to_index     —— 单词到索引的映射字典
#   C                 —— 分类类别数量，这里为 5（表示 5 种表情）
# 返回：
#   model —— 初始化好的 PyTorch 模型对象
#
# .to(device) —— 将模型转移到设备（device）上运行：
#   - 若检测到 GPU，则模型将在 CUDA 上训练；
#   - 若没有 GPU，则自动使用 CPU。
model = EmojifyV2(maxLen, word_to_vec_map, word_to_index, C=C).to(device)


# 小结：
# -------------------------------------------------
# 这一步完成了模型的核心初始化，包含：
#   - 嵌入层（加载预训练 GloVe 词向量）
#   - 双层 LSTM 网络结构
#   - Dropout 防止过拟合
#   - 全连接层 + Softmax 输出表情类别
#
# 模型此时已准备好进入训练阶段。


In [31]:
# ========================================
# 打印模型结构及统计参数
# ========================================

# -------------------------------------------------
# 打印模型的网络结构
# -------------------------------------------------
# print(model) 会输出模型各层的详细信息，包括：
#   - 各层类型（Embedding、LSTM、Dropout、Linear 等）
#   - 输入输出维度
#   - 可训练参数数量（trainable）等
print(model)


# -------------------------------------------------
# 统计模型总参数数量
# -------------------------------------------------
# sum(p.numel() for p in model.parameters())
#   - model.parameters() 返回模型中所有参数的迭代器
#   - p.numel() 统计张量 p 的元素总数
#   - sum(...) 对所有参数求和，即得到模型总参数数量
total_params = sum(p.numel() for p in model.parameters())


# -------------------------------------------------
# 统计可训练参数数量（trainable parameters）
# -------------------------------------------------
# sum(p.numel() for p in model.parameters() if p.requires_grad)
#   - p.requires_grad 为 True 表示该参数在反向传播时会更新
#   - 统计所有可训练参数数量，便于了解模型学习能力
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


# -------------------------------------------------
# 统计不可训练参数数量（non-trainable parameters）
# -------------------------------------------------
# total_params - trainable_params
#   - 这部分参数通常是固定的，比如冻结的嵌入层（freeze=True）
non_trainable_params = total_params - trainable_params


# -------------------------------------------------
# 打印参数统计结果
# -------------------------------------------------
# 使用千分位分隔符“,”格式化输出，更直观
print(f"Total params: {total_params:,}")          # 模型总参数
print(f"Trainable params: {trainable_params:,}")  # 可训练参数
print(f"Non-trainable params: {non_trainable_params:,}")  # 不可训练参数


EmojifyV2(
  (embedding): Embedding(400001, 50)
  (lstm1): LSTM(50, 128, batch_first=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (lstm2): LSTM(128, 128, batch_first=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)
Total params: 20,224,951
Trainable params: 224,901
Non-trainable params: 20,000,050


现在是训练模型的时候了。  

在 **X_train_indices** 和 **Y_train_oh** 上训练Pytorch模型。  
训练参数设置为：
- `epochs = 50`  
- `batch_size = 32`


In [33]:
# ========================================
# 训练 EmojifyV2 模型
# ========================================

# 调用训练函数 train(...) 来训练模型
# ----------------------------------------
# 参数说明：
# model               : PyTorch 模型实例，这里是 EmojifyV2
# X_train_indices     : 训练集输入的索引矩阵（每个句子转成 word index），形状 (m, maxLen)
# Y_train_oh          : 训练集标签的 one-hot 编码，形状 (m, C)
# num_epochs=50       : 总训练轮数，即完整遍历训练集 50 次
# batch_size=32       : 每个 batch 的样本数（梯度更新频率）
train(model, X_train_indices, Y_train_oh, num_epochs=50, batch_size=32)


Epoch 1/50
5/5 [==============================] - loss: 1.3565 - acc: 0.5076


Epoch 2/50
5/5 [==============================] - loss: 1.3698 - acc: 0.4924


Epoch 3/50
5/5 [==============================] - loss: 1.4118 - acc: 0.4924


Epoch 4/50
5/5 [==============================] - loss: 1.3630 - acc: 0.5227


Epoch 5/50
5/5 [==============================] - loss: 1.3264 - acc: 0.5076


...

Epoch 46/50
5/5 [==============================] - loss: 0.9852 - acc: 0.9091


Epoch 47/50
5/5 [==============================] - loss: 1.0258 - acc: 0.8939


Epoch 48/50
5/5 [==============================] - loss: 0.9692 - acc: 0.9242


Epoch 49/50
5/5 [==============================] - loss: 1.0219 - acc: 0.9091


Epoch 50/50
5/5 [==============================] - loss: 0.9637 - acc: 0.9318



每次运行准确率会有差异。  

运行以下代码单元，在测试集上评估你的模型表现。


In [34]:
# ========================================
# 测试集评估模型
# ========================================

# 调用 evaluate(...) 函数在测试集上评估模型性能
# ----------------------------------------
# 参数说明：
# model               : 已训练好的 PyTorch 模型实例（EmojifyV2）
# X_test_indices      : 测试集输入的索引矩阵（每个句子转成 word index），形状 (m_test, maxLen)
# Y_test_oh           : 测试集标签的 one-hot 编码，形状 (m_test, C)
# 返回值：
# pred_labels         : 模型预测标签，numpy 数组，形状 (m_test,)
pred_labels = evaluate(model, X_test_indices, Y_test_oh)


Test accuracy = 0.75


In [35]:
# ========================================
# 🖨 打印每个测试样本的预测结果
# ========================================

# 调用 print_predictions(...) 函数，将模型对测试集的预测结果与真实标签进行对比打印
# ----------------------------------------
# 参数说明：
# model            : 已训练好的 PyTorch 模型实例（EmojifyV2）
# X_test           : 测试集句子数组，形状 (m_test,) 或列表
# X_test_indices   : 测试集句子索引矩阵，形状 (m_test, maxLen)
# Y_test           : 测试集标签数组（整数形式），形状 (m_test,)
# label_to_emoji   : 函数，将标签整数映射为对应的 emoji 字符串

print_predictions(model, X_test, X_test_indices, Y_test, label_to_emoji)


错误信息如下（仅展示预测错误的样本）:
------------------------------------------------------------
✅ 正确表情：:smile:   ❌ 预测结果：he got a very nice raise	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：she got me a nice present	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：he is a good friend	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：We had such a lovely dinner tonight	 ❤️
✅ 正确表情：:disappointed:   ❌ 预测结果：This girl is messing with me	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：you brighten my day	 ❤️
✅ 正确表情：:disappointed:   ❌ 预测结果：she is a bully	 ❤️
✅ 正确表情：:disappointed:   ❌ 预测结果：My life is so boring	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：she said yes	 ❤️
✅ 正确表情：:smile:   ❌ 预测结果：will you be my valentine	 ❤️
✅ 正确表情：❤️   ❌ 预测结果：I love you to the stars and back	 ⚾
✅ 正确表情：:smile:   ❌ 预测结果：What you did was awesome	 :disappointed:
✅ 正确表情：:disappointed:   ❌ 预测结果：go away	 ⚾
✅ 正确表情：❤️   ❌ 预测结果：family is all I have	 :disappointed:



运行下方代码单元，查看被错误标注的示例。


现在你可以在自己的示例上尝试。  
在下面写下你自己的句子。


In [36]:
# ---------------------------- 单句预测 ---------------------------- #
def predict_single_sentence(model, sentence, word_to_index, maxLen, label_to_emoji):
    """
    单句预测函数
    
    功能：
    给定一条自然语言句子，利用训练好的 Emojify 模型预测对应的表情标签，
    并输出预测结果。
    
    参数说明：
    - model: 训练好的 PyTorch 模型
    - sentence: 待预测的输入句子（字符串）
    - word_to_index: 单词到索引的映射字典
    - maxLen: 每条句子填充后的最大长度（与训练阶段保持一致）
    - label_to_emoji: 将类别索引转换为表情符号的函数
    """

    # ① 将输入句子转为索引序列（符合模型输入格式）
    # sentences_to_indices() 的输入需要是一个 numpy 数组
    X_indices = sentences_to_indices(np.array([sentence]), word_to_index, maxLen)

    # ② 设置模型为评估模式（关闭 dropout 等）
    model.eval()

    # ③ 将索引数据转换为 PyTorch Tensor 并移动至设备（GPU 或 CPU）
    X_tensor = torch.tensor(X_indices, dtype=torch.long).to(device)

    # ④ 禁止梯度计算，加快预测速度
    with torch.no_grad():
        # 前向传播得到预测输出（每个类别的概率）
        pred = model(X_tensor)

        # 取概率最高的类别编号作为最终预测标签
        pred_label = torch.argmax(pred, dim=1).item()

    # ⑤ 打印预测结果（原句 + 对应表情）
    print(sentence + ' ' + label_to_emoji(pred_label))


In [37]:
# ========================================
# 单句预测
# ========================================

# 调用 predict_single_sentence(...) 函数，对单条句子进行预测
# ----------------------------------------
# 参数说明：
# model          : 已训练好的 PyTorch 模型实例（EmojifyV2）
# sentence       : 待预测的单条句子（字符串类型）
# word_to_index  : 字典，将词映射为对应的索引，用于 embedding 层输入
# maxLen         : 句子最大长度（padding 长度），与模型初始化时保持一致
# label_to_emoji : 函数，将标签整数映射为对应的 emoji 字符串

predict_single_sentence(model, "you are so beautiful", word_to_index, maxLen, label_to_emoji)


you are so beautiful ❤️


之前，Emojify-V1 模型无法正确标注 "*not feeling happy*"，但我们实现的 Emojifier-V2 能正确识别。（Keras 的输出每次略有随机性，因此你可能没有得到完全相同的结果。）  

当前模型在理解否定词（如 "*not happy*"）方面仍不够稳健，因为训练集较小，缺少否定句的示例。  
但如果训练集更大，LSTM 模型在理解这种复杂句子方面将远优于 Emojify-V1 模型。


### 恭喜！

你已经完成了本笔记本的全部内容！❤️❤️❤️

<font color='blue'>

**本次内容需要记住的要点（PyTorch 版本）**：

- 如果你的 NLP 任务训练集较小，使用词向量（word embeddings）可以显著提升算法效果。词向量使模型能够处理测试集中可能未出现在训练集中的单词。  
- 在 PyTorch 中训练序列模型，需要注意以下几点：
    - 使用 mini-batch 时，需要对序列进行填充（padding），确保 mini-batch 中的所有样本长度相同。  
    - `nn.Embedding` 层可以用预训练的词向量初始化（`nn.Embedding.from_pretrained()`）。这些词向量可以保持固定（`freeze=True`），也可以在你的数据集上进一步训练。但如果标注数据集很小，通常不值得训练大规模预训练词向量。  
    - `nn.LSTM` 的参数 `batch_first=True` 可保证输入输出形状为 `(batch, seq_len, features)`，返回值中包含隐藏状态和单个时间步输出。  
    - 可以在 `nn.LSTM` 后使用 `nn.Dropout` 来对网络进行正则化。  
    - 前向传播时，通常只取最后一个时间步的隐藏状态作为整个序列的表示，再传入 `nn.Linear` 输出类别概率。


恭喜你完成了本次作业并成功构建了一个 Emojifier。  
希望你对自己在本笔记本中的成果感到满意！

# 😀😀😀😀😀😀


## 致谢

感谢 Alison Darcy 和 Woebot 团队。  
Woebot 是一个可以随时与你交流的聊天机器人朋友，全天 24 小时在线。  
作为 Woebot 技术的一部分，它使用词向量来理解你所说内容的情绪。  
你可以访问 [http://woebot.io](http://woebot.io) 体验它。

<img src="images/woebot.png" style="width:600px;height:300px;">
